In [1]:
import requests
import pandas as pd
import numpy as np
import re

In [2]:
def get_name(sailors):
    names = list()
    if len(sailors) > 0:
        for sailor in sailors:
            names.append(sailor['name'])

        if len(names) == 1:
            name = names[0]
        else:
            name = ''
            for i in range(len(names) - 1):
                name += names[i]
                name += ', '
            name += names[-1]
        
        
        return name
    else:
        return np.nan

In [3]:
def get_position(text):
    pattern = r'\((.*?)\)'
    match = re.search(pattern, text)
    return match.group(1)

In [4]:
def get_flotilha(soup_object):
    """
    This functions gets a input like <span style="display:none;">punt_Yellow _0000003</span>
    and returns the series of this data.
    """
    if "Yellow" in str(soup_object) or "Blue" in str(soup_object) or "Red"in str(soup_object) or "Pink" in str(soup_object) or "White" in str(soup_object) or "Green" in str(soup_object) or "Emerald" in str(soup_object) \
    or "punt_ " in str(soup_object) or "Gold" in str(soup_object) or "Silver" in str(soup_object) or "Bronze" in str(soup_object):
        return "G"
    #DESCOMENTING BELOW IF THE CATEGORY IS FORMULA KITE
    # elif "QF" in str(soup_object):
    #     return "B"
    # elif "SF" in str(soup_object):
    #     return "P"
    # elif "Final" in str(soup_object):
    #     return "O"
    elif "Medal" in str(soup_object):
        return "MR"
    else:
        return "aa"

In [5]:
competitions = pd.read_excel("./Clusters-Sumulas.xlsx", sheet_name="Cluster 14")
competitions.head()

,Classe,Campeonato,Ano,Link,Descrição,Competição
0,Ilca 7,Semaine Olympique Francaise De Voile,2015,https://world-sailing-api.soticcloud.net/stats...,NaN,Semaine Olympique Francaise De Voile 2015
1,Ilca 6,Semaine Olympique Francaise De Voile,2015,https://world-sailing-api.soticcloud.net/stats...,NaN,Semaine Olympique Francaise De Voile 2015
2,49er,Semaine Olympique Francaise De Voile,2015,https://world-sailing-api.soticcloud.net/stats...,NaN,Semaine Olympique Francaise De Voile 2015
3,49erFX,Semaine Olympique Francaise De Voile,2015,https://world-sailing-api.soticcloud.net/stats...,NaN,Semaine Olympique Francaise De Voile 2015
4,Nacra 17,Semaine Olympique Francaise De Voile,2015,https://world-sailing-api.soticcloud.net/stats...,NaN,Semaine Olympique Francaise De Voile 2015


In [6]:
for i, competition in competitions.iterrows():
    
    COMPETICAO = competition["Competição"]
    CATEGORIA = competition["Classe"]

    r = requests.get(str(competition["Link"]))
    data = r.json()
    df_data = {
        "Id Resultado": [],
        "Id Competidor": [],
        "Nome Competidor": [],
        "Id Competição": [],
        "Classe Vela": [],
        "Pontuação Regata": [],
        "Flotilha": [],
        "Posição Geral": [],
        "Punição": [],
        "Pontuação Total": [],
        "Nett": [],
        "Nome Competição": [],
        "Regata": [],
        "Descarte": []    
    }

    for race in data["data"]["races"]:
        for result in race["raceResults"]:
            pontuacao = result["averagePoints"]
            nome = get_name(result["crew"]["sailors"])
            posicao = get_position(result["crew"]["_admin_name"])
            flotilha = get_flotilha(race["name"])
            totalpoints = result["crew"]["totalPoints"]
            net = result["crew"]["points"]
            flotilha = get_flotilha(race)
            regata = race["name"]
            descarte = 1 if result["discarded"] == "Y" else 0
            df_data["Id Resultado"].append(None)
            df_data["Id Competidor"].append(None),
            df_data["Nome Competidor"].append(nome),
            df_data["Id Competição"].append(None),
            df_data["Classe Vela"].append(CATEGORIA),
            df_data["Pontuação Regata"].append(pontuacao),
            df_data["Posição Geral"].append(posicao),
            df_data["Flotilha"].append(flotilha)
            df_data["Punição"].append(None),
            df_data["Pontuação Total"].append(totalpoints),
            df_data["Nett"].append(net),
            df_data["Nome Competição"].append(COMPETICAO)
            df_data["Regata"].append(regata)
            df_data['Descarte'].append(descarte)

    df = pd.DataFrame(df_data)

    if len(df) == 0:
        print(f'Erro {COMPETICAO} {CATEGORIA}')
    else:
        # df.head()
        df.to_csv(f"../scraped-data-2024/c14/{COMPETICAO} {CATEGORIA}.csv", index=False)
        print(f'Finished {COMPETICAO} {CATEGORIA}')


Finished Semaine Olympique Francaise De Voile 2015 Ilca 7
Finished Semaine Olympique Francaise De Voile 2015 Ilca 6
Finished Semaine Olympique Francaise De Voile 2015 49er
Finished Semaine Olympique Francaise De Voile 2015 49erFX
Finished Semaine Olympique Francaise De Voile 2015 Nacra 17
Finished Semaine Olympique Francaise De Voile 2016 Ilca 7
Finished Semaine Olympique Francaise De Voile 2016 Ilca 6
Finished Semaine Olympique Francaise De Voile 2016 49er
Finished Semaine Olympique Francaise De Voile 2016 49erFX
Finished Semaine Olympique Francaise De Voile 2016 Nacra 17
Erro Semaine Olympique Francaise De Voile 2019 Ilca 7
Erro Semaine Olympique Francaise De Voile 2019 Ilca 6
Erro Semaine Olympique Francaise De Voile 2022 Ilca 7
Erro Semaine Olympique Francaise De Voile 2022 Ilca 6
Erro Semaine Olympique Francaise De Voile 2022 49er
Erro Semaine Olympique Francaise De Voile 2022 49erFX
Erro Semaine Olympique Francaise De Voile 2022 Nacra 17
Erro Semaine Olympique Francaise De Voile 